In [ ]:
class NuScenesMTPInferenceDataset(torch.utils.data.Dataset):
    """
    Dataset equivalente al de entrenamiento pero sin GT.
    Permite generar predicciones para cada instancia del split.
    """
    def __init__(self, split, helper, input_representation, device='cpu'):
        self.split = split
        self.helper = helper
        self.input_representation = input_representation
        self.device = device

    def __len__(self):
        return len(self.split)

    def __getitem__(self, idx):
        instance_token, sample_token = self.split[idx].split("_")

        # Imagen rasterizada igual que en train
        img = self.input_representation.make_input_representation(instance_token, sample_token)
        img_tensor = torch.Tensor(img).permute(2,0,1).to(self.device)

        # Estado del agente (vel, acc, heading-change-rate)
        vel = self.helper.get_velocity_for_agent(instance_token, sample_token)
        acc = self.helper.get_acceleration_for_agent(instance_token, sample_token)
        hcr = self.helper.get_heading_change_rate_for_agent(instance_token, sample_token)

        agent_state = torch.Tensor([vel, acc, hcr])
        agent_state = torch.nan_to_num(agent_state, nan=0.0).to(self.device)

        return img_tensor, agent_state, instance_token, sample_token


In [ ]:
import json
import numpy as np
from tqdm import tqdm
from nuscenes.eval.prediction.data_classes import Prediction

def generate_submission_notebook(model, dataset, output_path="submission.json"):
    model.eval()
    predictions_list = []

    print("🚗 Ejecutando inferencia sobre todo el split...")
    
    for i in tqdm(range(len(dataset))):
        img, agent_state, instance_token, sample_token = dataset[i]

        img = img.unsqueeze(0)          # batch=1
        agent_state = agent_state.unsqueeze(0)

        with torch.no_grad():
            pred = model(img, agent_state)

        # pred shape → [1, num_modes, 24]
        num_modes = pred.shape[1]
        pred = pred.reshape(num_modes, 12, 2).cpu().numpy()

        # Si solo tienes 1 modo → prob = [1.0]
        probs = np.ones(num_modes) / num_modes

        prediction_obj = Prediction(
            instance=instance_token,
            sample=sample_token,
            prediction=pred,
            probabilities=probs
        )

        predictions_list.append(prediction_obj.serialize())

    with open(output_path, "w") as f:
        json.dump(predictions_list, f, indent=2)

    print(f"💾 Submission guardado en {output_path}")
    return output_path


In [ ]:
from nuscenes import NuScenes
from nuscenes.prediction import PredictHelper
from nuscenes.eval.prediction.config import load_prediction_config
from nuscenes.eval.prediction.compute_metrics import compute_metrics

def evaluate_submission_notebook(version, dataroot, submission_path,
                                 config_name="predict_2020_icra.json"):
    
    print("📊 Cargando nuScenes y configuración de métricas...")
    nusc = NuScenes(version=version, dataroot=dataroot)
    helper = PredictHelper(nusc)
    config = load_prediction_config(helper, config_name)

    print("📥 Cargando predicciones...")
    predictions = json.load(open(submission_path, "r"))

    print("🏁 Calculando métricas...")
    results = compute_metrics(predictions, helper, config)

    output_metrics = submission_path.replace(".json", "_metrics.json")
    json.dump(results, open(output_metrics, "w"), indent=2)

    print(f"📄 Métricas guardadas en {output_metrics}")
    print("\n📌 Resultados:")
    display(results)

    return results


In [ ]:
# Crear dataset de inferencia
dataset_val = NuScenesMTPInferenceDataset(mini_val, helper, mtp_input_representation, device=device)

# Generar submission
submission_path = generate_submission_notebook(model, dataset_val, "submission_mtp.json")

# Evaluar
metrics = evaluate_submission_notebook("v1.0-mini", dataroot, submission_path)


# EXPLORACIÓ DE LES DADES

In [ ]:
from nuscenes import NuScenes

dataroot = "/data/sets/nuscenes"   # ajusta si es distinto
nusc = NuScenes(version="v1.0-mini", dataroot=dataroot)

print("Número de escenas:", len(nusc.scene))
print("Número de anotaciones:", len(nusc.sample_annotation))
print("Número de muestras (keyframes):", len(nusc.sample))
print("Número de instancias:", len(nusc.instance))

In [ ]:
import os

version_dir = os.path.join(dataroot, "v1.0-mini")
json_files = [f for f in os.listdir(version_dir) if f.endswith(".json")]

print("Archivos JSON disponibles en v1.0-mini:\n")
for f in json_files:
    print(" -", f)


In [ ]:
import json

def show_json_keys(filename, n=5):
    print(f"\nContenido parcial de {filename}:")
    path = os.path.join(version_dir, filename)
    data = json.load(open(path))
    for key in data[0]:
        print(" •", key)
    print(f"Ejemplo:\n{data[0]}")
    
show_json_keys("sample.json")
show_json_keys("sample_annotation.json")
show_json_keys("instance.json")
show_json_keys("ego_pose.json")


Visualitzar una escena amb sensors

In [ ]:
nusc.render_sample_data(nusc.sample[0]['data']['CAM_FRONT'])
nusc.render_sample_data(nusc.sample[0]['data']['LIDAR_TOP'])
nusc.render_sample_data(nusc.sample[0]['data']['RADAR_FRONT'])


Visualitzar mapa drivable area

In [ ]:
from nuscenes.map_expansion.map_api import NuScenesMap

sample_token = nusc.sample[0]['token']
log = nusc.get('log', nusc.get('scene', nusc.sample[0]['scene_token'])['log_token'])
map_name = log['location']
nmap = NuScenesMap(dataroot=dataroot, map_name=map_name)

nmap.render_map(channel="drivable_area")


Trajectoria pasada i futura d'un agent

In [ ]:
from nuscenes.prediction import PredictHelper
helper = PredictHelper(nusc)

# Cogemos el primer agente de la primera muestra
sample_token = nusc.sample[0]['token']
anns = nusc.sample[0]['anns']
instance_token = nusc.get('sample_annotation', anns[0])['instance_token']

past = helper.get_past_for_agent(instance_token, sample_token, 2, in_agent_frame=False)
future = helper.get_future_for_agent(instance_token, sample_token, 6, in_agent_frame=False)

import matplotlib.pyplot as plt

plt.figure(figsize=(6,6))
plt.plot(past[:,0], past[:,1], 'bo-', label="Pasado")
plt.plot(future[:,0], future[:,1], 'ro-', label="Futuro (GT)")
plt.legend()
plt.title("Trayectoria del agente (pasado y futuro)")
plt.grid()
plt.axis('equal')
plt.show()


Visualitzar rasterización que usa el model MTP

In [ ]:
img = mtp_input_representation.make_input_representation(instance_token, sample_token)
plt.figure(figsize=(5,5))
plt.imshow(img)
plt.title("Raster utilizado como input del modelo MTP")
plt.axis('off')
plt.show()


Dades que utilitza com a tal el model

In [ ]:
vel = helper.get_velocity_for_agent(instance_token, sample_token)
acc = helper.get_acceleration_for_agent(instance_token, sample_token)
hcr = helper.get_heading_change_rate_for_agent(instance_token, sample_token)

print("Velocidad:", vel)
print("Aceleración:", acc)
print("Rate cambio heading:", hcr)


In [ ]:
plt.imshow(img)
plt.title("Entrada visual REAL del modelo MTP")
plt.axis('off')
plt.show()


In [ ]:
plt.plot(future[:,0], future[:,1], 'ro-')
plt.title("Trayectoria Ground Truth usada para entrenar MTP")
plt.grid()
plt.axis('equal')
plt.show()
